<a href="https://colab.research.google.com/github/RunBadger/Biomass-modeling-and-mapping/blob/main/Biomass_modeling_and_mapping_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pBjS5VG0kgsKIxwt6rKLkU8GT6FR_C6XwndHFBBMXA8&tc=Ja_fVS-wwLsGZQ2GknfQ8nrewaI5xJsDwUeMVum_FCI&cc=moQOKaV9yMs0laaB3DhmIJkvrBV5MXKoegHLtOel_ZY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJq8Q1ATiFpKnnewbXg6yrv54ySTw7_q88gUB1EVH6ETVgc_XHpy_5o

Successfully saved authorization token.


In [ ]:
import tensorflow as tf

# Installs geemap package
import subprocess
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) 
import geemap
print(geemap.__version__)

0.17.1


# Define variables

In [ ]:
# Your Earth Engine username. Cloud assets contained shapefiles (with labels) and DEM data.
USER_NAME = 'ee-my-duosong'
# Cloud Storage bucket
OUTPUT_BUCKET = 'ee-regression-demos'

# Prepare images

In [ ]:
import geemap.colormaps as cm
import numpy as np
Map = geemap.Map(center=(34.3921,-258.5573), zoom=10.5)

# A rectangle region, covering the whole study area, is necessary for model prediction.
region=ee.FeatureCollection('projects/ee-my-duosong/assets/Region')
Map.addLayer(region, {}, 'Region')

nlist  =  [ '201906',   '201908',      '201909'  ]
startdate = [ '2019-06-12', '2019-07-27',   '2019-09-05'  ]
enddate =  [ '2019-06-13', '2019-07-28',  '2019-09-06' ]
sentinel_images=[]

for i in range(0,3):
  sentinel = ee.ImageCollection("COPERNICUS/S2").filterBounds(region) \
  .filterDate(startdate[i], enddate[i]) \
  .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
  .mosaic() \
  .clip(region)\
  .select(['B4', 'B3', 'B2','B8']) 
  sentinel_images.append(sentinel)
        
image_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
sentinel_images

In [ ]:
k=4
Map.addLayer(sentinel_images[k], image_vis, nlist[k])
Map

Map(center=[34.3921, -258.5573], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [ ]:
def NDVI(image):
  ndvi=image.normalizedDifference(['B8','B4'])
  return ndvi
ranges=range(0,7)
ndvi_list=[]
for i in ranges:
  nd=NDVI(sentinel_images[i])
  ndvi_list.append(nd)

palette = cm.palettes.ndvi
prediction_vis = {
  'min': 0,
  'max': 1,
  'palette': palette}

In [ ]:
# DEM, slop and aspect
dem = ee.Image('projects/ee-my-duosong/assets/DEM_Region')
# Map.addLayer(dem, vis_params, "DuosongDEM")
slope = ee.Terrain.slope(dem)
# Map.addLayer(slope, {min: 0, max :60}, 'slope')
# import math
aspect = ee.Terrain.aspect(dem)
# sinImage = aspect.divide(180).multiply(math.pi).sin()
# Map.addLayer(sinImage, {min: -1, max: 1}, 'aspect')

In [ ]:
OT_0=dem.expression( 'float(B*0)',
    {'B':dem.select(['b1'])} )
OT_1=dem.expression( 'float(B*0)+1',
    {'B':dem.select(['b1'])} )

OT_June = OT_1.rename('June').addBands(OT_0.rename('July')).addBands(OT_0.rename('Aug')).addBands(OT_0.rename('Sep'))
OT_Aug = OT_0.rename('June').addBands(OT_0.rename('July')).addBands(OT_1.rename('Aug')).addBands(OT_0.rename('Sep'))
OT_Sep = OT_0.rename('June').addBands(OT_0.rename('July')).addBands(OT_0.rename('Aug')).addBands(OT_1.rename('Sep'))
OT_Month=[OT_June, OT_Aug, OT_Sep]

Temp=[ '5.17' ,'9.26'  , '9.16'  ]
Rain=[ '1.83' ,'4.92'  , '2.23'  ]
MT=[]

for k in range(0,3):
  M=OT_0.expression('B+'+ Temp[k],{'B':OT_0.select(['b1'])}).rename('temp')\
  .addBands(OT_0.expression( 'B+'+ Rain[k],{'B':OT_0.select(['b1'])}).rename('rain')).divide(10)
  MT.append(M)
# Map.addLayer(MT[0], {}, 'OT')
# Map

# Image Normalization 

In [ ]:
# // Function to Normalize Image
# // Pixel Values should be between 0 and 1
# // Formula is (x - xmin) / (xmax - xmin)

def normalize(image):
  bandNames = image.bandNames()
# // Compute min and max of the image
  minDict = image.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry=region,
    scale= 10,
    maxPixels= 1e9,
    bestEffort= True,
    tileScale=16   )
  maxDict = image.reduceRegion(
    reducer= ee.Reducer.max(),
    geometry= region,
    scale= 10,
    maxPixels=1e9,
    bestEffort=True,
    tileScale= 16   )
  mins = ee.Image.constant(minDict.values(bandNames))
  maxs = ee.Image.constant(maxDict.values(bandNames))
  normalized = image.subtract(mins).divide(maxs.subtract(mins))
  return normalized

In [ ]:
# Combining images
comb_images=[]
for i in ranges:
  image = normalize(sentinel_images[i].addBands(slope).addBands(aspect).addBands(dem.rename('elevation'))\
          .addBands(ndvi_list[i]) ).addBands(OT_Month[i]).addBands(MT[i])
  comb_images.append(image)

# Map.addLayer(comb_images[1], {}, 'comb_images')
# image_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}

In [ ]:
comb_images

In [ ]:
# image_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
# Map.addLayer(comb_images[6], image_vis, 'comb_images')
# Map

In [ ]:
watershed=ee.FeatureCollection('projects/ee-my-duosong/assets/watershed')
Map.addLayer(watershed, {}, 'Watershed')

AGB201906=ee.FeatureCollection('projects/ee-my-duosong/assets/AGB_201906')
AGB201908=ee.FeatureCollection('projects/ee-my-duosong/assets/AGB_201907')
AGB201909=ee.FeatureCollection('projects/ee-my-duosong/assets/AGB_201909')
LABEL_AGB=[AGB201906, AGB201908,  AGB201909]
LABEL = 'Biomass'

# Map.addLayer(AGB201906, {}, 'AGB201906')
# Map

In [ ]:
# Sample the image at the points and add a random column.
samples=[]
trainings=[]
testings=[]
ranges=range(0,3)
for i in ranges:
  sample = comb_images[i].sampleRegions(
    collection=LABEL_AGB[i], properties=[LABEL], scale=10).randomColumn()
  # Partition the sample approximately 70-30.
  training = sample.filter(ee.Filter.lt('random', 0.7))
  testing = sample.filter(ee.Filter.gte('random', 0.7))
  samples.append(sample)
  trainings.append(training)
  testings.append(testing)
samples

In [ ]:
from pprint import pprint
# Print the first couple points to verify.
pprint({'training': samples[1].first().getInfo()})
# pprint({'testing': trainings[0].first().getInfo()})

{'training': {'geometry': None,
              'id': '00000000000000000000_0',
              'properties': {'B2': 0.04194733500480652,
                             'B3': 0.047872625291347504,
                             'B4': 0.02804691530764103,
                             'B8': 0.25128427147865295,
                             'Biomass': 426.98475,
                             'aspect': 0.22753555342423207,
                             'elevation': 0.38693034648895264,
                             'nd': 0.860800627558854,
                             'rain': 0.492,
                             'random': 0.5894215157469578,
                             'slope': 0.17278192469217857,
                             'temp': 0.9259999999999999},
              'type': 'Feature'}}


In [ ]:
# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
BANDS = ['aspect',  'elevation', 'B2', 'B3', 'B4', 'B8', 'slope','nd', 'temp' , 'rain' ]
# 'B1','B10','B11','B12','B5','B6','B7', 'B8A', 'B9'
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)
# url = sample.getDownloadURL()
# print(url)

In [ ]:
# Make sure you can see the output bucket.  You must have write access.
import tensorflow as tf
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Can not find output Cloud Storage bucket.')

Found Cloud Storage bucket.


In [ ]:
# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = ['Training_201906','Training_201908','Training_201909']
TEST_FILE_PREFIX = ['Testing_201906','Testing_201908','Testing_201909']
file_extension = '.tfrecord.gz'

# Create the tasks.
for i in range(0,3):
  TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX[i] + file_extension
  TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX[i] + file_extension
  training_task = ee.batch.Export.table.toCloudStorage(
    collection=trainings[i],
    description=TRAIN_FILE_PREFIX[i],
    fileNamePrefix=TRAIN_FILE_PREFIX[i],
    bucket=OUTPUT_BUCKET,
    fileFormat='TFRecord',
    selectors=FEATURE_NAMES)

  testing_task = ee.batch.Export.table.toCloudStorage(
    collection=testings[i],
    description=TEST_FILE_PREFIX[i],
    fileNamePrefix=TEST_FILE_PREFIX[i],
    bucket=OUTPUT_BUCKET,
    fileFormat='TFRecord',
    selectors=FEATURE_NAMES)
  # Start the tasks.
  training_task.start()
  testing_task.start()

In [ ]:
# Print all tasks.
from pprint import pprint
pprint(ee.batch.Task.list()[0:9])

[<Task EFNHKILYZGORTA3QBEHJ7H24 EXPORT_FEATURES: Testing_201909 (COMPLETED)>,
 <Task EVSU3XXGWINR6CP3HOLDC7XW EXPORT_FEATURES: Training_201909 (COMPLETED)>,
 <Task BKVLBWOFJVQNBKXH32VYDBTN EXPORT_FEATURES: Testing_201908 (COMPLETED)>,
 <Task 472NRRXFRYTARTSSOKA5LEZU EXPORT_FEATURES: Training_201908 (COMPLETED)>,
 <Task BDTTE7SWZH5T4CSJKTMXRTDC EXPORT_FEATURES: Testing_201906 (COMPLETED)>,
 <Task P7EABCKE6PWWPRBEEZUGOUZJ EXPORT_FEATURES: Training_201906 (COMPLETED)>,
 <Task OQ5LLASPR5ZCKH4TFKE7RRGP EXPORT_IMAGE: Image_201909 (COMPLETED)>,
 <Task AZYLNXEPHJ4K2HXHRJ42VTFC EXPORT_IMAGE: Image_201908 (COMPLETED)>,
 <Task TWYUC3VHGGTMZKPLX2EUQZJA EXPORT_FEATURES: Testing_201909 (COMPLETED)>]


# Export the imagery
The export region has to be a rectangle.

It's also possible to monitor an individual task. Here we poll the task until it's done. If you do this, please put a sleep() in the loop to avoid making too many requests. Note that this will block until complete (you can always halt the execution of this cell).

In [ ]:
comb_images

In [ ]:
# Export imagery in this region. It must be a rectangle.
EXPORT_REGION = ee.Geometry.Rectangle([-258.80, 34.53, -258.29, 34.24])
# EXPORT_REGION_test = ee.Geometry.Rectangle([-258.639, 34.407, -258.502, 34.35])

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = ['Image_201906','Image_201908','Image_201909']

for i in range(0,3):
  # Specify patch and file dimensions.
  image_export_options = {
    'patchDimensions': [512, 512],
    'maxFileSize': 104857600,
    'compressed': True}

  image_task = ee.batch.Export.image.toCloudStorage(
    image=comb_images[i],
    description=IMAGE_FILE_PREFIX[i],
    fileNamePrefix=IMAGE_FILE_PREFIX[i],
    bucket=OUTPUT_BUCKET,
    scale=10,
    fileFormat='TFRecord',
    region=EXPORT_REGION.toGeoJSON()['coordinates'],
    formatOptions=image_export_options,  )
  # Start the task.
  image_task.start()
  import time
  while image_task.active():
    print('Polling for task (id: {}).'.format(image_task.id))
    time.sleep(30)
  print('Done with image export. '+ IMAGE_FILE_PREFIX[i])

Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Polling for task (id: NZBYM36XGVRBU3ZUDB2ZZKMO).
Done with image export. Image_201906
Polling for task (id: NUWFVJUF2N4UFJUZRP5TMBHD).
Polling for task (id: NUWFVJUF2N

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list()[0:10])

[<Task OF5USAT4ABTN6GCBN62353S6 EXPORT_IMAGE: Image_202109 (COMPLETED)>,
 <Task YIOFYRXL2LQY72VBYH5XSILC EXPORT_IMAGE: Image_202108 (COMPLETED)>,
 <Task LOYV2TXB7PUF6ZHC4PELX7KB EXPORT_IMAGE: Image_202009 (COMPLETED)>,
 <Task TCX4RWDQKU6YA2ULBZ3EMM3J EXPORT_IMAGE: Image_202008 (COMPLETED)>,
 <Task IM7GZ3WYZU3CTNLEBGVKDODN EXPORT_IMAGE: Image_201909 (COMPLETED)>,
 <Task JDLJKJTTFTFOGNTG7NUHW5A7 EXPORT_IMAGE: Image_201908 (COMPLETED)>,
 <Task HEVO3SM3TFOQMQHMLRFBGSWE EXPORT_IMAGE: Image_201906 (COMPLETED)>,
 <Task IS74NYPZV2DZEE34VPIBEQNH EXPORT_FEATURES: Testing_201909 (COMPLETED)>,
 <Task KP4DMDCB5FS7SI5Q5VYFZKOL EXPORT_FEATURES: Training_201909 (COMPLETED)>,
 <Task GC53GCZZ4UE4HXTB4RRJGG3X EXPORT_FEATURES: Testing_201908 (COMPLETED)>]


# Data preparation and pre-processing

In [ ]:
# # Create dataset from multiple .tfrecord files in Cloud Storage.
TRAIN_FILE_LIST=[]
TEST_FILE_LIST=[]
for i in ranges:
  TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX[i] + file_extension 
  TRAIN_FILE_LIST.append(TRAIN_FILE_PATH)
  TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX[i] + file_extension 
  TEST_FILE_LIST.append(TEST_FILE_PATH)


In [ ]:
TEST_FILE_LIST

['gs://ee-regression-demos/Testing_201906.tfrecord.gz',
 'gs://ee-regression-demos/Testing_201908.tfrecord.gz',
 'gs://ee-regression-demos/Testing_201909.tfrecord.gz']

In [ ]:
# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES ]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))
pprint(features_dict)

{'Aug': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B8': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'Biomass': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'July': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'June': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'Sep': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'aspect': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'elevation': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'nd': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'rain': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'slope': FixedLenFeature(s

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by featuresDict.
  Args:
    example_proto: a serialized Example.
  Returns:
    A tuple of the predictors dictionary and the label.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features,labels

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  (Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.)
def to_tuple(inputs,label):
  return (tf.transpose(list(inputs.values())), [label] ) 

Note that each record of the parsed dataset contains a tuple. The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values. The second element of the tuple is a class label.

In [ ]:
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_LIST, compression_type='GZIP')

In [ ]:
# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)
# Map the to_tuple function, shuffle and batch.
input_dataset = parsed_dataset.map(to_tuple).batch(5)
input_dataset
pprint(iter(input_dataset).next())

(<tf.Tensor: shape=(5, 1, 14), dtype=float32, numpy=
array([[[0.        , 0.04229952, 0.06553377, 0.03255844, 0.16461653,
         0.        , 1.        , 0.        , 0.22753556, 0.38693035,
         0.8364583 , 0.183     , 0.17278193, 0.517     ]],

       [[0.        , 0.03804883, 0.05333333, 0.02806763, 0.13255988,
         0.        , 1.        , 0.        , 0.17137593, 0.40240756,
         0.81526685, 0.183     , 0.22561249, 0.517     ]],

       [[0.        , 0.03789332, 0.05333333, 0.027077  , 0.126922  ,
         0.        , 1.        , 0.        , 0.19338277, 0.41530526,
         0.8116847 , 0.183     , 0.1535193 , 0.517     ]],

       [[0.        , 0.03934477, 0.06291939, 0.02747325, 0.18521108,
         0.        , 1.        , 0.        , 0.459269  , 0.41100603,
         0.87863207, 0.183     , 0.20433502, 0.517     ]],

       [[0.        , 0.04022601, 0.06143791, 0.02912429, 0.16615413,
         0.        , 1.        , 0.        , 0.4098187 , 0.4393809 ,
         0.853039

# Creat DNN model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics

# Define r square for accuracy assessment.
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu' ),
    layers.Dense(64, activation='relu' ),
    layers.Dense(64, activation='relu' ),
    layers.Dense(1)  ])
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae',r_square])
  return model
model = build_model()

In [ ]:
EPOCHS=100
history = model.fit(input_dataset, epochs=EPOCHS)

Epoch 1/100
76/76 [==============================] - 0s 4ms/step - loss: 24496.6855 - mae: 109.3717 - r_square: -0.6968
Epoch 2/100
76/76 [==============================] - 0s 4ms/step - loss: 24514.4199 - mae: 109.5854 - r_square: -0.6977
Epoch 3/100
76/76 [==============================] - 0s 4ms/step - loss: 24452.7617 - mae: 109.4709 - r_square: -0.7079
Epoch 4/100
76/76 [==============================] - 0s 4ms/step - loss: 24475.5586 - mae: 109.4211 - r_square: -0.6982
Epoch 5/100
76/76 [==============================] - 0s 4ms/step - loss: 24372.8125 - mae: 109.3472 - r_square: -0.6973
Epoch 6/100
76/76 [==============================] - 0s 4ms/step - loss: 24508.7324 - mae: 109.5003 - r_square: -0.7087
Epoch 7/100
76/76 [==============================] - 0s 4ms/step - loss: 24356.1367 - mae: 109.2621 - r_square: -0.7047
Epoch 8/100
76/76 [==============================] - 0s 4ms/step - loss: 24493.6191 - mae: 109.4929 - r_square: -0.6928
Epoch 9/100
76/76 [=====================

In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_LIST, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(250))
model.evaluate(test_dataset)

1/1 [==============================] - 0s 351ms/step - loss: 25769.4395 - mae: 109.4440 - r_square: 0.6265


[25769.439453125, 109.44404602050781, 0.6264588832855225]

In [ ]:
p=model.predict(test_dataset)
p

# **Use the trained model to estiamte an image from Earth Engine**

In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
IMAGE_FILE_PREFIX = ['Image_201906','Image_201908','Image_201909','Image_202008','Image_202009','Image_202108','Image_202109']
image_files_list = [[],[],[],[],[],[],[]]
json_file = [[],[],[],[],[],[],[]]
for i in range(0,7):
  exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX[i] in s]

  # Get the list of image files and the JSON mixer file.
  for f in exported_files_list:
    if f.endswith('.tfrecord.gz'):
      image_files_list[i].append(f)
    elif f.endswith('.json'):
      json_file[i].append(f)

  # Make sure the files are in the right order.
  image_files_list[i].sort()

# pprint(image_files_list)
print(json_file)

[['gs://ee-regression-demos/Image_201906mixer.json'], ['gs://ee-regression-demos/Image_201908mixer.json'], ['gs://ee-regression-demos/Image_201909mixer.json'], ['gs://ee-regression-demos/Image_202008mixer.json'], ['gs://ee-regression-demos/Image_202009mixer.json'], ['gs://ee-regression-demos/Image_202108mixer.json'], ['gs://ee-regression-demos/Image_202109mixer.json']]


In [ ]:
import json
mixer = []
for j in json_file:
  # Load the contents of the mixer file to a JSON object.
  json_text = !gsutil cat {j[0]}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  m = json.loads(json_text.nlstr)
  mixer.append(m)
  # pprint(m)
print(mixer)

[{'projection': {'crs': 'EPSG:4326', 'affine': {'doubleMatrix': [8.983152841195215e-05, 0.0, 101.19997782706993, 0.0, -8.983152841195215e-05, 34.530341206270286]}}, 'patchDimensions': [512, 512], 'patchesPerRow': 11, 'totalPatches': 66}, {'projection': {'crs': 'EPSG:4326', 'affine': {'doubleMatrix': [8.983152841195215e-05, 0.0, 101.19997782706993, 0.0, -8.983152841195215e-05, 34.530341206270286]}}, 'patchDimensions': [512, 512], 'patchesPerRow': 11, 'totalPatches': 66}, {'projection': {'crs': 'EPSG:4326', 'affine': {'doubleMatrix': [8.983152841195215e-05, 0.0, 101.19997782706993, 0.0, -8.983152841195215e-05, 34.530341206270286]}}, 'patchDimensions': [512, 512], 'patchesPerRow': 11, 'totalPatches': 66}, {'projection': {'crs': 'EPSG:4326', 'affine': {'doubleMatrix': [8.983152841195215e-05, 0.0, 101.19997782706993, 0.0, -8.983152841195215e-05, 34.530341206270286]}}, 'patchDimensions': [512, 512], 'patchesPerRow': 11, 'totalPatches': 66}, {'projection': {'crs': 'EPSG:4326', 'affine': {'dou

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer[0]['patchDimensions'][0]
patch_height = mixer[0]['patchDimensions'][1]
patches = mixer[0]['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS ]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))
# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

In [ ]:
image_datasets=[]
for n in range(0,7):
  # Note that you can make one dataset from many files by specifying a list.
  image_dataset = tf.data.TFRecordDataset(image_files_list[n], compression_type='GZIP')
  # Parse the data into tensors, one long tensor per patch.
  image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)
  # Break our long tensors into many little ones.
  image_dataset = image_dataset.flat_map(
    lambda features: tf.data.Dataset.from_tensor_slices(features)  )
  # Turn the dictionary in each record into a tuple without a label.
  image_dataset = image_dataset.map(
    lambda data_dict: (tf.transpose(list(data_dict.values())), )  )
  # Turn each patch into a batch.
  image_dataset = image_dataset.batch(patch_width * patch_height)
  image_datasets.append(image_dataset)
image_datasets

[<BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>,
 <BatchDataset shapes: ((None, 1, 14),), types: (tf.float32,)>]

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions=[]
for i in range(0,7):
  prediction = model.predict(image_datasets[i], steps=patches, verbose=1)
  predictions.append(prediction)
# Note that the predictions come as a numpy array.  Check the first one.
# print(predictions[0])

66/66 [==============================] - 499s 8s/step


In [ ]:
print(predictions[6])

[[[320.73407  ]]

 [[299.2942   ]]

 [[289.56952  ]]

 ...

 [[  7.1970177]]

 [[  7.197016 ]]

 [[  6.933075 ]]]


## Write the predictions to a TFRecord file and upload them to an Earth Engine asset

In [ ]:
IMAGE_PREDICT = ['Pred_201906','Pred_201908','Pred_201909','Pred_202008','Pred_202009','Pred_202108','Pred_202109']

for i in range(0,7):
  # The output path for the prediction image (i.e. predictions) TFRecord file.
  OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/'+ IMAGE_PREDICT[i] +'_demo.TFRecord'
  print('Writing to file ' + OUTPUT_IMAGE_FILE)
  # The name of the Earth Engine asset (legacy assets) to be created by importing
  # the predicted image from the TFRecord file in Cloud Storage.
  OUTPUT_ASSET_ID = 'users/' + 'Sentinel_2A_HenanC' + '/'+ IMAGE_PREDICT[i]

  # Instantiate the writer.
  writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

  # Every patch-worth of predictions we'll dump an example into the output
  # file with a single feature that holds our predictions. Since our predictions
  # are already in the order of the exported data, the patches we create here
  # will also be in the right order.
  patch = []
  cur_patch = 1
  for prediction in predictions[i]:
    patch.append(prediction[0][0])

    # Once we've seen a patches-worth of class_ids...
    if (len(patch) == patch_width * patch_height):
      print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
      # Create an example
      example = tf.train.Example(
        features=tf.train.Features(
          feature={ 'AGB': tf.train.Feature(
                float_list=tf.train.FloatList(
                    value=patch)),} ) )
      # Write the example to the file and clear our patch array so it's ready for
      # another batch of class ids
      writer.write(example.SerializeToString())
      patch = []
      cur_patch += 1
  writer.close()  
  print('..............')
  !gsutil ls -l {OUTPUT_IMAGE_FILE}
  # Start the upload.
  print('---- Uploading to ' + OUTPUT_ASSET_ID)
  !earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file[i][0]}
  print('------------------') 

Writing to file gs://ee-regression-demos/Pred_201906_demo.TFRecord
Done with patch 1 of 66...
Done with patch 2 of 66...
Done with patch 3 of 66...
Done with patch 4 of 66...
Done with patch 5 of 66...
Done with patch 6 of 66...
Done with patch 7 of 66...
Done with patch 8 of 66...
Done with patch 9 of 66...
Done with patch 10 of 66...
Done with patch 11 of 66...
Done with patch 12 of 66...
Done with patch 13 of 66...
Done with patch 14 of 66...
Done with patch 15 of 66...
Done with patch 16 of 66...
Done with patch 17 of 66...
Done with patch 18 of 66...
Done with patch 19 of 66...
Done with patch 20 of 66...
Done with patch 21 of 66...
Done with patch 22 of 66...
Done with patch 23 of 66...
Done with patch 24 of 66...
Done with patch 25 of 66...
Done with patch 26 of 66...
Done with patch 27 of 66...
Done with patch 28 of 66...
Done with patch 29 of 66...
Done with patch 30 of 66...
Done with patch 31 of 66...
Done with patch 32 of 66...
Done with patch 33 of 66...
Done with patch 34

In [ ]:
ee.batch.Task.list()[0:9]

[<Task R3M3KAMJ5RNDO773LXPCYAS6 INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202109" (COMPLETED)>,
 <Task YLLZXTMDYLLZUNAKT4GYXUZW INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202108" (COMPLETED)>,
 <Task TL6RDOFOQ3JRGQWSQZ5XPVIK INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202009" (COMPLETED)>,
 <Task QJZE2AEYPBX6UL4QBZCVBIXX INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202008" (COMPLETED)>,
 <Task B3UTTHG32HVKNMVIRVOLHDTS INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_201909" (COMPLETED)>,
 <Task ZF4EE7PQAAUNAYYUVEKCKKXZ INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_201908" (COMPLETED)>,
 <Task FXEOWEDUMR7AVHUGUH4TOUE7 INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2

# View the ingested asset

In [ ]:
predictions_images=[]
IMAGE_PREDICT = ['Pred_201906','Pred_201908','Pred_201909','Pred_202008','Pred_202009','Pred_202108','Pred_202109']
for i in range(0,7):
  OUTPUT_ASSET_ID = 'users/' + 'Sentinel_2A_HenanC' + '/'+ IMAGE_PREDICT[i]
  predictions_image = ee.Image(OUTPUT_ASSET_ID)
  predictions_images.append(predictions_image)
  geemap.ee_export_image_to_drive(predictions_image, description=IMAGE_PREDICT[i], 
                    folder='export',  scale=10)

Exporting Pred_201906 ...
Exporting Pred_201908 ...
Exporting Pred_201909 ...
Exporting Pred_202008 ...
Exporting Pred_202009 ...
Exporting Pred_202108 ...
Exporting Pred_202109 ...


In [ ]:
ee.batch.Task.list()[0:9]

[<Task BKG23KE5YY4FMX2TGKN4ZP5A EXPORT_IMAGE: Pred_202109 (COMPLETED)>,
 <Task FTX3IYU33A7BTTMYONXS6ZSC EXPORT_IMAGE: Pred_202108 (COMPLETED)>,
 <Task RS7ZSLEN7CW4P5FUJVTRN4XS EXPORT_IMAGE: Pred_202009 (COMPLETED)>,
 <Task VFD2EJ3WRKPYUMBUPV73BE6N EXPORT_IMAGE: Pred_202008 (COMPLETED)>,
 <Task XKR5P3QUOLKSG25YII6CNHEL EXPORT_IMAGE: Pred_201909 (COMPLETED)>,
 <Task 3LUP5MDS7FUZYQVYMCHH7E4Y EXPORT_IMAGE: Pred_201908 (COMPLETED)>,
 <Task ZT7CU6STHTTSGW4C2PITQQE6 EXPORT_IMAGE: Pred_201906 (COMPLETED)>,
 <Task R3M3KAMJ5RNDO773LXPCYAS6 INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202109" (COMPLETED)>,
 <Task YLLZXTMDYLLZUNAKT4GYXUZW INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/Sentinel_2A_HenanC/Pred_202108" (COMPLETED)>]

In [ ]:
Map = geemap.Map(center=(34.3921,-258.5573), zoom=10.5)

In [ ]:

palette = cm.palettes.ndvi
prediction_vis = {
  'min': 000,
  'max': 1000,
  'palette': palette}
Map.addLayer(predictions_images[0],prediction_vis,nlist[0])
# Map.addLayer(predictions_images[1],prediction_vis,nlist[1])
Map

Map(center=[34.3921, -258.5573], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [ ]:
OUTPUT_ASSET_ID_T = 'users/' + 'Sentinel_2A_HenanC' + '/Classified_pixel_demo'

In [ ]:
predictions_image_T = ee.Image(OUTPUT_ASSET_ID_T)
prediction_vis_T = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']}
Map.addLayer(predictions_image_T,prediction_vis_T,'CCC')
Map